In [ ]:
!wget https://hf.co/danjacobellis/walloc/resolve/main/Stereo_Li_192c_J8_nf8_v1.0.2.pth

In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

In [1]:
import torch
import numpy as np
import torchaudio
import tempfile
from demucs.separate import Separator
from IPython.display import Audio
from datasets import load_dataset
from walloc import walloc
from spauq.core.metrics import spauq_eval
from fastprogress.fastprogress import progress_bar
class Config: pass

In [2]:
device = "cuda"
checkpoint = torch.load("Stereo_Li_192c_J8_nf8_v1.0.2.pth",map_location="cpu",weights_only=False)
codec_config = checkpoint['config']
codec = walloc.Codec1D(
    channels = codec_config.channels,
    J = codec_config.J,
    Ne = codec_config.Ne,
    Nd = codec_config.Nd,
    latent_dim = codec_config.latent_dim,
    latent_bits = codec_config.latent_bits,
    lightweight_encode = codec_config.lightweight_encode,
    post_filter = codec_config.post_filter
).to(device)
codec.load_state_dict(checkpoint['model_state_dict'])
codec.eval();

/home/dgj335/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [3]:
MUSDB = load_dataset("danjacobellis/musdb18HQ", split='validation')

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

In [4]:
separator = Separator()

In [5]:
def pad(audio, p=2**16):
    B,C,L = audio.shape
    padding_size = (p - (L % p)) % p
    if padding_size > 0:
        audio = torch.nn.functional.pad(audio, (0, padding_size), mode='constant', value=0)
    return audio

In [6]:
max_duration=44100*180
SSDR = []
SRDR = []
with torch.no_grad():
    for i_sample in progress_bar(range(0, len(MUSDB), 5)):
        SSDR.append([])
        SRDR.append([])
        y = []
        for i_instr in range(5):
            sample = MUSDB[i_sample+i_instr]
            instr = sample['instrument']
            x, fs = torchaudio.load(sample['audio']['bytes'])
            x = x[:,:max_duration]
            L = x.shape[-1]
            x_padded = pad(x.unsqueeze(0), 2**16).to(device)

            X = codec.wavelet_analysis(x_padded,codec.J)
            Y = codec.encoder(X)
            X_hat = codec.decoder(Y)
            x_hat = codec.wavelet_synthesis(X_hat,codec.J)
            x_hat = codec.post(x_hat)

            if i_instr == 0:
                pred = separator.separate_tensor(x_hat[0])
            else:
                SDR = spauq_eval(
                    reference=x_padded[0].to("cpu"),
                    estimate=pred[1][instr].to("cpu"),
                    fs = fs
                )
                SSDR[-1].append(SDR['SSR'])
                SRDR[-1].append(SDR['SRR'])
SSDR = torch.tensor(SSDR)
SRDR = torch.tensor(SRDR)

/home/dgj335/.local/lib/python3.10/site-packages/spauq/core/preprocessing.py:325: UserWarning: No forgive_mode specified, defaulting to `none`
  warnings.warn(


In [7]:
SSDR.mean(dim=0)

tensor([14.0237, 17.3669, 15.4579, 11.6876], dtype=torch.float64)

In [8]:
SSDR.mean()

tensor(14.6340, dtype=torch.float64)

In [9]:
SRDR.mean(dim=0)

tensor([ 4.3192,  7.3617,  0.3864, -1.6658], dtype=torch.float64)

In [11]:
SRDR.mean()

tensor(2.6004, dtype=torch.float64)